# **IP4_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Variables:
1. S_Target: Target sales (in units)
2. S_Actual: Actual sales (in units)
3. S_PosDev: Positive Deviation in sales (S_Actual - S_Target)
4. S_NegDev: Negative Deviation in sales (S_Target - S_Actual)

5. FT_H_Target: Target hours for full-time employees 
6. FT_H_Actual: Actual hours worked by full-time employees
7. FT_H_PosDev: Positive Deviation in full-time hours (FT_H_Actual - FT_H_Target)
8. FT_H_NegDev: Negative Deviation in full-time hours (FT_H_Target - FT_H_Actual)

9. PT_H_Target: Target hours for part-time employees 
10. PT_H_Actual: Actual hours worked by part-time employees
11. PT_H_PosDev: Positive Deviation in part-time hours (PT_H_Actual - PT_H_Target)
12. PT_H_NegDev: Negative Deviation in part-time hours (PT_H_Target - PT_H_Actual)

13. FT_OvertimeTarget: Target overtime hours for full-time employees
14. FT_OvertimeActual: Actual overtime hours worked by full-time employees
15. FT_OvertimePosDev: Positive Deviation in full-time overtime (FT_OvertimeActual - 

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:
Minimize
P1*(S_PosDev + S_NegDev) + 
P2*(FT_OvertimePosDev + FT_OvertimeNegDev) + 
2*P3*(FT_H_PosDev + FT_H_NegDev) + 
P3*(PT_H_PosDev + PT_H_NegDev) +
3*P4*(PT_OvertimeActual) +
P4*(FT_OvertimePosDev + FT_OvertimeNegDev)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. S_PosDev >= S_Actual - 5500: This sets the minimum positive deviation in sales.
2. S_NegDev >= 5500 - S_Actual: This sets the minimum negative deviation in sales.

3. FT_H_PosDev >= FT_H_Actual - 800: Minimum positive deviation in full-time hours.
4. FT_H_NegDev >= 800 - FT_H_Actual: Minimum negative deviation in full-time hours.

5. PT_H_PosDev >= PT_H_Actual - 320: Minimum positive deviation in part-time hours.
6. PT_H_NegDev >= 320 - PT_H_Actual: Minimum negative deviation in part-time hours.

7. FT_OvertimePosDev >= FT_OvertimeActual - 100: Minimum positive deviation in full-time overtime.
8. FT_OvertimeNegDev >= 100 - FT_OvertimeActual: Minimum negative deviation in full-time overtime.

9. S_Actual = (5 * FT_H_Actual) + (2 * PT_H_Actual): The actual sales depend on the number of hours worked by full-time and part-time employees.

10. FT_H_Actual = FT_H_Target + FT_OvertimeActual: Full-time hours actually worked is the target worked hours plus any overtime
11. PT_H

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response4.choices[0].message.content)

Here is the Pyomo model corresponding to the problem:

```python
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Sample data
S_Target = 5500
FT_H_Target = 800
PT_H_Target = 320
FT_OvertimeTarget = 100
P = [1, 5, 3, 10]  # Priority factors

# Create a model
model = ConcreteModel()

# Define variables
model.S_Act = Var(within=NonNegativeReals)
model.FT_H_Act = Var(within=NonNegativeReals)
model.PT_H_Act = Var(within=NonNegativeReals)
model.FT_OvertimeAct = Var(within=NonNegativeReals)
model.PT_OvertimeAct = Var(within=NonNegativeReals)
model.S_PosDev = Var(within=NonNegativeReals)
model.S_NegDev = Var(within=NonNegativeReals)
model.FT_H_PosDev = Var(within=NonNegativeReals)
model.FT_H_NegDev = Var(within=NonNegativeReals)
model.PT_H_PosDev = Var(within=NonNegativeReals)
model.PT_H_NegDev = Var(within=NonNegativeReals)
model.FT_OvertimePosDev = Var(within=NonNegativeReals)
model.FT_OvertimeNegDev = Var(within=NonNegativeReals)

# Objective function
model.obj = Objective(expr = P[0]*(model.S_PosDev + model.S_NegDev) +
                           P[1]*(model.FT_OvertimePosDev + model.FT_OvertimeNegDev) +
                           2*P[2]*(model.FT_H_PosDev + model.FT_H_NegDev) +
                           P[2]*(model.PT_H_PosDev + model.PT_H_NegDev) +
                           3*P[3]*(model.PT_OvertimeAct) +
                           P[3]*(model.FT_OvertimePosDev + model.FT_OvertimeNegDev),
                      sense=minimize)

# Constraints
model.Sales_Positive_Dev_Constraint = Constraint(expr = model.S_PosDev >= model.S_Act - S_Target)
model.Sales_Negative_Dev_Constraint = Constraint(expr = model.S_NegDev >= S_Target - model.S_Act)
model.Full_Time_Positive_Dev_Constraint = Constraint(expr = model.FT_H_PosDev >= model.FT_H_Act - FT_H_Target)
model.Full_Time_Negative_Dev_Constraint = Constraint(expr = model.FT_H_NegDev >= FT_H_Target - model.FT_H_Act)
model.Part_Time_Positive_Dev_Constraint = Constraint(expr = model.PT_H_PosDev >= model.PT_H_Act - PT_H_Target)
model.Part_Time_Negative_Dev_Constraint = Constraint(expr = model.PT_H_NegDev >= PT_H_Target - model.PT_H_Act)
model.Full_Time_Overtime_Positive_Dev_Constraint = Constraint(expr = model.FT_OvertimePosDev >= model.FT_OvertimeAct - FT_OvertimeTarget)
model.Full_Time_Overtime_Negative_Dev_Constraint = Constraint(expr = model.FT_OvertimeNegDev >= FT_OvertimeTarget - model.FT_OvertimeAct)
model.Sales_Constraint = Constraint(expr = model.S_Act == (5 * model.FT_H_Act) + (2 * model.PT_H_Act))
model.Full_Time_Actual_Constraint = Constraint(expr = model.FT_H_Act == FT_H_Target + model.FT_OvertimeAct)
model.Part_Time_Actual_Constraint = Constraint(expr = model.PT_H_Act == PT_H_Target + model.PT_OvertimeAct)

# Solve
SolverFactory('glpk').solve(model)

# print the results
print("Actual Sales: ", model.S_Act())
print("Full-Time Actual Hours: ", model.FT_H_Act())
print("Part-Time Actual Hours: ", model.PT_H_Act())
print("Full-Time Overtime: ", model.FT_OvertimeAct())
print("Part-Time Overtime: ", model.PT_OvertimeAct())
```
Please replace sample data with your own data to solve the problem. This code finds the optimal sales and hours (both regular and overtime) for full-time and part-time employees that minimizes deviation from target values prioritized by different penalty factors, given the constraints. Please note that the number of units each employee is able to produce per hour is assumed to be 5 for full-time employees and 2 for part-time employees.

### **Run the code Generated by GPT4**

In [15]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Sample data
S_Target = 5500
FT_H_Target = 800
PT_H_Target = 320
FT_OvertimeTarget = 100
P = [1, 5, 3, 10]  # Priority factors

# Create a model
model = ConcreteModel()

# Define variables
model.S_Act = Var(within=NonNegativeReals)
model.FT_H_Act = Var(within=NonNegativeReals)
model.PT_H_Act = Var(within=NonNegativeReals)
model.FT_OvertimeAct = Var(within=NonNegativeReals)
model.PT_OvertimeAct = Var(within=NonNegativeReals)
model.S_PosDev = Var(within=NonNegativeReals)
model.S_NegDev = Var(within=NonNegativeReals)
model.FT_H_PosDev = Var(within=NonNegativeReals)
model.FT_H_NegDev = Var(within=NonNegativeReals)
model.PT_H_PosDev = Var(within=NonNegativeReals)
model.PT_H_NegDev = Var(within=NonNegativeReals)
model.FT_OvertimePosDev = Var(within=NonNegativeReals)
model.FT_OvertimeNegDev = Var(within=NonNegativeReals)

# Objective function
model.obj = Objective(expr = P[0]*(model.S_PosDev + model.S_NegDev) +
                           P[1]*(model.FT_OvertimePosDev + model.FT_OvertimeNegDev) +
                           2*P[2]*(model.FT_H_PosDev + model.FT_H_NegDev) +
                           P[2]*(model.PT_H_PosDev + model.PT_H_NegDev) +
                           3*P[3]*(model.PT_OvertimeAct) +
                           P[3]*(model.FT_OvertimePosDev + model.FT_OvertimeNegDev),
                      sense=minimize)

# Constraints
model.Sales_Positive_Dev_Constraint = Constraint(expr = model.S_PosDev >= model.S_Act - S_Target)
model.Sales_Negative_Dev_Constraint = Constraint(expr = model.S_NegDev >= S_Target - model.S_Act)
model.Full_Time_Positive_Dev_Constraint = Constraint(expr = model.FT_H_PosDev >= model.FT_H_Act - FT_H_Target)
model.Full_Time_Negative_Dev_Constraint = Constraint(expr = model.FT_H_NegDev >= FT_H_Target - model.FT_H_Act)
model.Part_Time_Positive_Dev_Constraint = Constraint(expr = model.PT_H_PosDev >= model.PT_H_Act - PT_H_Target)
model.Part_Time_Negative_Dev_Constraint = Constraint(expr = model.PT_H_NegDev >= PT_H_Target - model.PT_H_Act)
model.Full_Time_Overtime_Positive_Dev_Constraint = Constraint(expr = model.FT_OvertimePosDev >= model.FT_OvertimeAct - FT_OvertimeTarget)
model.Full_Time_Overtime_Negative_Dev_Constraint = Constraint(expr = model.FT_OvertimeNegDev >= FT_OvertimeTarget - model.FT_OvertimeAct)
model.Sales_Constraint = Constraint(expr = model.S_Act == (5 * model.FT_H_Act) + (2 * model.PT_H_Act))
model.Full_Time_Actual_Constraint = Constraint(expr = model.FT_H_Act == FT_H_Target + model.FT_OvertimeAct)
model.Part_Time_Actual_Constraint = Constraint(expr = model.PT_H_Act == PT_H_Target + model.PT_OvertimeAct)

# Solve
SolverFactory('glpk').solve(model)

# print the results
print("Actual Sales: ", model.S_Act())
print("Full-Time Actual Hours: ", model.FT_H_Act())
print("Part-Time Actual Hours: ", model.PT_H_Act())
print("Full-Time Overtime: ", model.FT_OvertimeAct())
print("Part-Time Overtime: ", model.PT_OvertimeAct())

Actual Sales:  5140.0
Full-Time Actual Hours:  900.0
Part-Time Actual Hours:  320.0
Full-Time Overtime:  100.0
Part-Time Overtime:  0.0


In [17]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Sample data
S_Target = 5500
FT_H_Target = 800
PT_H_Target = 320
FT_OvertimeTarget = 100
P = [4,3,2,1]  # Priority factors

# Create a model
model = ConcreteModel()

# Define variables
model.S_Act = Var(within=NonNegativeReals)
model.FT_H_Act = Var(within=NonNegativeReals)
model.PT_H_Act = Var(within=NonNegativeReals)
model.FT_OvertimeAct = Var(within=NonNegativeReals)
model.PT_OvertimeAct = Var(within=NonNegativeReals)
model.S_PosDev = Var(within=NonNegativeReals)
model.S_NegDev = Var(within=NonNegativeReals)
model.FT_H_PosDev = Var(within=NonNegativeReals)
model.FT_H_NegDev = Var(within=NonNegativeReals)
model.PT_H_PosDev = Var(within=NonNegativeReals)
model.PT_H_NegDev = Var(within=NonNegativeReals)
model.FT_OvertimePosDev = Var(within=NonNegativeReals)
model.FT_OvertimeNegDev = Var(within=NonNegativeReals)

# Objective function
model.obj = Objective(expr = P[0]*(model.S_PosDev + model.S_NegDev) +
                           P[1]*(model.FT_OvertimePosDev + model.FT_OvertimeNegDev) +
                           2*P[2]*(model.FT_H_PosDev + model.FT_H_NegDev) +
                           P[2]*(model.PT_H_PosDev + model.PT_H_NegDev) +
                           3*P[3]*(model.PT_OvertimeAct) +
                           P[3]*(model.FT_OvertimePosDev + model.FT_OvertimeNegDev),
                      sense=minimize)

# Constraints
model.Sales_Constraint = Constraint(expr = model.S_Act == (5 * model.FT_H_Act) + (2 * model.PT_H_Act))
model.Sales_Positive_Dev_Constraint = Constraint(expr = model.S_PosDev >= model.S_Act - S_Target)
model.Sales_Negative_Dev_Constraint = Constraint(expr = model.S_NegDev >= S_Target - model.S_Act)
model.Full_Time_Positive_Dev_Constraint = Constraint(expr = model.FT_H_PosDev >= model.FT_H_Act - FT_H_Target)
model.Full_Time_Negative_Dev_Constraint = Constraint(expr = model.FT_H_NegDev >= FT_H_Target - model.FT_H_Act)
model.Part_Time_Positive_Dev_Constraint = Constraint(expr = model.PT_H_PosDev >= model.PT_H_Act - PT_H_Target)
model.Part_Time_Negative_Dev_Constraint = Constraint(expr = model.PT_H_NegDev >= PT_H_Target - model.PT_H_Act)
model.Full_Time_Overtime_Positive_Dev_Constraint = Constraint(expr = model.FT_OvertimePosDev >= model.FT_OvertimeAct - FT_OvertimeTarget)
model.Full_Time_Overtime_Negative_Dev_Constraint = Constraint(expr = model.FT_OvertimeNegDev >= FT_OvertimeTarget - model.FT_OvertimeAct)
model.Full_Time_Actual_Constraint = Constraint(expr = model.FT_H_Act == FT_H_Target + model.FT_OvertimeAct)
model.Part_Time_Actual_Constraint = Constraint(expr = model.PT_H_Act == PT_H_Target + model.PT_OvertimeAct)

# Solve
SolverFactory('glpk').solve(model)

# print the results
print("Actual Sales: ", model.S_Act())
print("Full-Time Actual Hours: ", model.FT_H_Act())
print("Part-Time Actual Hours: ", model.PT_H_Act())
print("Full-Time Overtime: ", model.FT_OvertimeAct())
print("Part-Time Overtime: ", model.PT_OvertimeAct())
print(model.obj())

Actual Sales:  5500.0
Full-Time Actual Hours:  972.0
Part-Time Actual Hours:  320.0
Full-Time Overtime:  172.0
Part-Time Overtime:  -0.0
976.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**